# Script to download Posters


In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from PIL import Image
from io import BytesIO

# 🔹 Replace with your actual TMDb API key
TMDB_API_KEY = "123"

# 🔹 Read the movie dataset (ensure it contains a 'title' column)
data = pd.read_csv("tmdb_5000_movies.csv")

# 🔹 Directory to save posters
POSTER_DIR = "posters"
os.makedirs(POSTER_DIR, exist_ok=True)

# 🔹 Function to get the poster URL from TMDb
def get_poster_url(movie_title):
    url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={movie_title}"
    response = requests.get(url).json()
    
    if response["results"]:
        poster_path = response["results"][0].get("poster_path", None)
        if poster_path:
            return f"https://image.tmdb.org/t/p/w500{poster_path}"  # w500 = medium resolution
    return None  # No poster found

# 🔹 Function to download and save the poster
def download_poster(movie_id, movie_title):
    poster_url = get_poster_url(movie_title)
    if poster_url:
        response = requests.get(poster_url)
        image = Image.open(BytesIO(response.content))
        image_path = os.path.join(POSTER_DIR, f"{movie_id}.jpg")  # Save as movie_id.jpg
        image.save(image_path)
        return True
    return False

# 🔹 Loop through the dataset and download posters
for _, row in tqdm(data.iterrows(), total=len(data)):
    movie_id, title = row["id"], row["title"]
    success = download_poster(movie_id, title)
    if success:
        print(f"✅ Downloaded: {title}")
    else:
        print(f"❌ No poster found: {title}")

print("🎬 All posters downloaded successfully!")